## 水力模型计算

In [2]:
import numpy as np
from scipy.optimize import newton
from scipy.linalg import solve
from math import pi, log10

# 常数设定
N, Np, Nd = 3, 3, 2
D = np.ones((3, 1))*150e-3  # 管道直径
ep = np.ones((3, 1))*1.25e-3  # 管道粗糙度
s = pi*D*D/4  # 管道横截面面积
len = np.array([[400], [400], [600]])  # 管道长度
mq = np.array([[2], [3]])  # 节点质量流量 kg/s
rho = 958.4  # 100°C 水的密度 (kg/m^3) 
g = 9.81  # 重力加速度
viscosity = 0.294e-6  # 温度为100℃，单位: m^2/s 动力粘度
A = np.array([[1, -1, 0], [0, 1, 1], [-1, 0, -1]])  # 网络节点-弧关联矩阵
B = np.array([[1, 1, -1]])
dm = np.array([[1], [1], [1]])  # 初始化dm
err = 1
pre = 0

# Colebrook公式
def colebrook(R, K):
    F_init = 0.02
    return newton(lambda F: 1/np.sqrt(F) + 2*log10(K/3.7 + 2.51/(R*np.sqrt(F))), F_init)

while err > 1e-3:
    # 计算管道流量dm     
    m_node = A.dot(dm)  # 节点流入量
    dPhi = m_node[:N-1] - mq  # 节点流量误差
    HJ0 = A[:N-1,:]   
    # 对于环路类型热力网
    vel = dm.flatten()/s.flatten()/rho  # 单位m kg/s, V m/s
    Re = abs(vel)*D.flatten()/viscosity
    factor = np.array([colebrook(Re[i], ep[i]/D[i]) if Re[i] >= 2300 else 64/Re[i] for i in range(Np)])
    Kf = factor*len.flatten()/D.flatten()/s.flatten()/s.flatten()/2/g/rho/rho
    dpre = B.dot(Kf*abs(dm.flatten())*dm.flatten())  # loop方程的压力
    HJpre = 2*B*(Kf*abs(dm.flatten()))  # loop压力方程的雅可比矩阵
    dH = np.vstack((dPhi, dpre))
    HJ = np.block([[HJ0], [HJpre]])  
    dx = solve(HJ, -dH)  # 解HJ*dx = -dH
    err = max(abs(dH))
    dm = dm + dx
    pre = pre + 1
   
print(f"每根管道内的质量流量:\n{dm}")


每根管道内的质量流量:
[[2.71151397]
 [0.71151397]
 [2.28848603]]


## 根据水力模型求解稳态热力温度(供水温度)

In [53]:
# 热力模型参数设定
Ts3 = 100  # 热源供水温度
Ta = 10    # 环境温度
Ts3d = Ts3 - Ta  # 热源供水与环境温差
lam = 0.2  # 传热系数 W / (m * K)
cp = 4182  # 水的比容 J / (kg * K)
To = np.array([50, 50])  # 负荷热量温度
Tod = To - Ta
# 计算每个节点的实际通过流量
mq = A.dot(dm).flatten()

# 根据水力模型所算出的管道流量求出每段管路的温降系数 psi
psi = np.exp(-1 * lam * len.flatten() / (cp * dm.flatten()))

# 联立供水温度节点方程组
# 系数矩阵
Cs = np.array([
    [1, 0],
    [-1 * dm[1, 0] * psi[1], mq[1]]
])

# 列向量
bs = Ts3d * np.array([psi[0], dm[2, 0] * psi[2]])

# 求出对应的供水温度
Tsx = solve(Cs, bs) + Ta

# 打印结果
print(f'节点1的供水温度为: {Tsx[0]:.2f}℃')
print(f'节点2的供水温度为: {Tsx[1]:.2f}℃')



节点1的供水温度为: 99.37℃
节点2的供水温度为: 98.43℃


### 继续计算回水温度

In [67]:
# 建立联立回水温度节点方程组
Cr = np.array([
    [dm[0,0], -1 * dm[1,0] * psi[1]], 
    [0, 1]
])

br = Tod * np.array([mq[0],1])

# 求出对应的回水温度
Trxd = solve(Cr, br)

# 计算热源节点3的回水温度
Tr3 = np.array([
    [dm[0,0] * psi[0], dm[2,0] * psi[2]] 
]).dot(Trxd) / (- mq[2]) + Ta

print(f'节点3的回水温度为: {Tr3[0]:.2f}℃')

节点3的回水温度为: 49.47℃
